## Testing the emulation

This notebook will include one of our networks, and run the inference. Then, the same model will be tested from its hls version, to check that the emulation works. 

For the qkeras and hls4ml tests, this will be based on our framework. For the emulation, this will be based on the "quickstart.py" file that is doing the same for the VAE from the anomaly team.

In [1]:
### first, lets set all required environment variables

! source /scratch/Xilinx/Vivado/2018.3/settings64.sh
! export XILINX_HLS=/scratch/Xilinx/Vivado/2018.3 

In [2]:
# this will include the emulation
# strangely, the first one will crash whith a symbol error
try: import topotrigger_emulation as emulation
except Exception as e: print(repr(e))
    
import topotrigger_emulation as emulation

In [3]:
# and then the framework stuff

# just suppressing the CUDA warnings from keras
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np

# importing the required modules
import sys
sys.path.insert(1, '/scratch/flabe/uhh_mlatl1_run3/src')

from loadData import *
from preprocessing import *
from models import *
from training import *
from utils import *
from plotting import *
from inference import *
from tests import *

from tensorflow import keras
from sklearn.utils import shuffle

set_global_plotting_settings()

/scratch/uh2ml4l1/anaconda3/envs/finns_framework_setup/lib/python3.7/site-packages/awkward0/__init__.py:8: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(numpy.__version__) < distutils.version.LooseVersion("1.13.1"):


In [4]:
### definitions

keras_model = "/scratch/flabe/" + "hhtobbww/models/" + "onlymu_small_compressed_4folds_hw_bitshiftscaler"